In [ ]:
# default_exp data.gaussian_dg

In [ ]:
# export
from scipy.stats import norm, uniform,dirichlet
import numpy as np
import scipy.stats as stats
import pdb as pdb
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from random import seed, randint, random
from sklearn.datasets import make_spd_matrix as spd
from sklearn import metrics
from scipy.stats import multivariate_normal as mvn
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from types import SimpleNamespace as SN


In [ ]:
# export
class mixture:

    def __init__(self, components, mixing_proportion):
        self.comps = components
        self.mixProp = mixing_proportion
        # self.sigmoid = activations.sigmoid()

    def pdf(self, x):
        return np.sum((p * comp.pdf(x) for (comp, p) in zip(self.comps, self.mixProp)), axis=1)

    def cdf(self, x):
        return np.sum((p * comp.cdf(x) for (comp, p) in zip(self.comps, self.mixProp)), axis=1)

    def rvs(self, size):
        sizes = np.cast['int32'](np.floor(size * self.mixProp))
        #pdb.set_trace()
        delta = np.cast['int32'](size - np.sum(sizes))
        ix = np.random.choice(np.size(self.mixProp), size=delta, p=self.mixProp)
        for ii in ix:
            sizes[ii] = sizes[ii] + 1
        dim = np.size(self.comps[0].rvs(size=1))
        x = np.empty([0, dim])
        for (s, comp) in zip(sizes, self.comps):
            new = comp.rvs(size=[s, 1])
            if len(new.shape) == 1:
                new = np.expand_dims(new,1)
            x = np.concatenate((x,new), axis=0)
        #pdb.set_trace()
        return x

    def rvsCompInfo(self, size):
        sizes = np.cast['int32'](np.floor(size * self.mixProp))
        #pdb.set_trace()
        delta = np.cast['int32'](size - np.sum(sizes))
        ix = np.random.choice(np.size(self.mixProp), size=delta, p=self.mixProp)
        for ii in ix:
            sizes[ii] = sizes[ii] + 1
        dim = np.size(self.comps[0].rvs(size=1))
        x = np.empty([0, dim])
        y = np.empty([0, 1])
        k = 0
        for (s, comp) in zip(sizes, self.comps):
            new = comp.rvs(size=[s, 1])
            print(x.shape, new.shape)
            x = np.concatenate((x, new), axis=0)
            y = np.concatenate((y, np.zeros([s, 1]) + k), axis=0)
            k = k + 1
        #pdb.set_trace()
        return x, y

    def component_pdfs(self, x):
        return (comp.pdf(x) for comp in self.comps)

In [ ]:
# export
class DataGenerator:

    def __init__(self, dist_p, dist_n, alpha, n_p, n_u, batch_size=1024):
        self.dist_p = dist_p
        self.dist_n = dist_n
        self.alpha = alpha
        self.n_p = n_p
        self.n_u = n_u
        self.n_up = np.cast['int32'](np.floor(n_u * alpha))
        self.n_un = self.n_u - self.n_up
        self.batch_size = batch_size

    @classmethod
    def _shuffle_in_unison(self, a, b):
        p = np.random.permutation(len(a))
        return a[p],b[p]
        
    def data_pos(self, n):
        #pdb.set_trace()
        return np.reshape(self.dist_p.rvs(size=n), newshape=(n, -1))


    def data_neg(self, n):
        return np.reshape(self.dist_n.rvs(size=n), newshape=(n, -1))

    def data_pos_compInfo(self, n):
        #pdb.set_trace()
        x, c = self.dist_p.rvsCompInfo(size=n)
        x = np.reshape(x, newshape=(n, -1))
        return x, c

    def data_neg_compInfo(self, n):
        x, c = self.dist_n.rvsCompInfo(size=n)
        x = np.reshape(x, newshape=(n, -1))
        return x, c

    def data_ul(self, n, alpha):
        n_up = np.cast['int32'](np.floor(n * alpha))
        n_un = n - n_up
        x_up = self.data_pos(n_up)
        x_un = self.data_neg(n_un)
        x = np.concatenate((x_up, x_un), axis=0)
        y = np.zeros([n, 1])
        y[np.arange(x_up.shape[0]), 0] = 1
        return x, y

    def pu_data(self):
        x_p = self.data_pos(self.n_p)
        x_u, y_u = self.data_ul(self.n_u, self.alpha)
        x_pu = np.concatenate((x_p, x_u), axis=0)
        y_pu = np.zeros([x_pu.shape[0], 1])
        y_pu[np.arange(self.n_p), 0] = 1
        y_pn = np.concatenate((np.ones([self.n_p, 1]), y_u), axis=0)
        # y_pn = y_pu
        # y_pn[x_pu.size(0):(self.n_p - 1):-1, 0] = y_u
        return x_pu, y_pu, y_pn
        
    def pn_data(self, n, alpha):
        "return x,y"
        x_p = self.data_pos(int(alpha*n))
        x_n = self.data_neg(int((1-alpha)*n))
        y_p = np.ones((len(x_p), 1))
        y_n = np.zeros((len(x_n), 1))
        x = np.vstack((x_p, x_n))
        y = np.vstack((y_p, y_n))
        x, y = self._shuffle_in_unison(x, y)
        return x, y

    def pn_data_compInfo(self, n, alpha):
        x_p, c_p = self.data_pos_compInfo(int(alpha*n))
        x_n, c_n = self.data_neg_compInfo(int((1-alpha)*n))
        y_p = np.ones((len(x_p), 1))
        y_n = np.zeros((len(x_n), 1))
        x = np.vstack((x_p, x_n))
        y = np.vstack((y_p, y_n))
        c = np.vstack((c_p, c_n))
        return x, y, c
    
    def dens_pos(self, x):
        return self.dist_p.pdf(x)

    def dens_neg(self, x):
        return self.dist_n.pdf(x)

    def dens_mix(self, x, a):
        return a * self.dens_pos(x) + (1 - a) * self.dens_neg(x)

    def pn_posterior(self, x, a):
        return a * self.dens_pos(x) / self.dens_mix(x, a)

    def pu_posterior(self, x):
        c1 = self.n_p / (self.n_u + self.n_p)
        c2 = (self.n_up + self.n_p) / (self.n_u + self.n_p)
        return c1 * self.dens_pos(x) / self.dens_mix(x, c2)

    def pn_posterior_sts(self, x):
        c = (self.n_up + self.n_p) / (self.n_u + self.n_p)
        return self.pn_posterior(x, c)

    def pn_posterior_cc(self, x):
        return self.pn_posterior(x, self.alpha)

    def pn_posterior_balanced(self, x):
        return self.pn_posterior(x, 0.5)


class GaussianDG(DataGenerator):

    def __init__(self, mu, sig, alpha, n_p, n_u, batch_size=1024):
        dist_p = norm(loc=0, scale=1)
        dist_n = norm(loc=mu, scale=sig)
        super(GaussianDG, self).__init__(dist_p=dist_p, dist_n=dist_n, alpha=alpha, n_p=n_p, n_u=n_u, batch_size=batch_size)


class UniformDG(DataGenerator):

    def __init__(self, mu, sig, alpha, n_p, n_u, batch_size=1024):
        dist_p = uniform(loc=0, scale=1)
        dist_n = uniform(loc=mu, scale=sig)
        super(UniformDG, self).__init__(dist_p=dist_p, dist_n=dist_n, alpha=alpha, n_p=n_p, n_u=n_u, batch_size=batch_size)


class NormalMixDG(DataGenerator):

    def __init__(self, mu_pos, sig_pos, p_pos, mu_neg, sig_neg, p_neg, alpha, n_pos, n_ul, batch_size = 1024):
        components_pos = [norm(loc=mu, scale=sig) for (mu, sig) in zip(mu_pos, sig_pos)]
        components_neg = [norm(loc=mu, scale=sig) for (mu, sig) in zip(mu_neg, sig_neg)]
        dist_pos = mixture(components_pos, p_pos)
        dist_neg = mixture(components_neg, p_neg)
        super(NormalMixDG, self).__init__(dist_p=dist_pos, dist_n=dist_neg, alpha=alpha, n_p=n_pos, n_u=n_ul, batch_size=batch_size)


class MVNormalMixDG(DataGenerator):

    def __init__(self, mu_pos, sig_pos, p_pos, mu_neg, sig_neg, p_neg, alpha, n_pos, n_ul, batch_size = 1024):
        self.components_pos = [mvn(mean=mu, cov=sig) for (mu, sig) in zip(mu_pos, sig_pos)]
        self.components_neg = [mvn(mean=mu, cov=sig) for (mu, sig) in zip(mu_neg, sig_neg)]
        dist_pos = mixture(self.components_pos, p_pos)
        dist_neg = mixture(self.components_neg, p_neg)
        super(MVNormalMixDG, self).__init__(dist_p=dist_pos, dist_n=dist_neg, alpha=alpha, n_p=n_pos, n_u=n_ul, batch_size=batch_size)




In [ ]:
# export
class NormalMixRandomParameters:

    def __init__(self, dim, max_comps):
        self.dim = dim
        self.max_comps = max_comps
        self.n_comps_pos = randint(1, max_comps)
        self.n_comps_neg = randint(1, max_comps)
        self.mu_pos = [np.array([2 * random() - 1 for i in np.arange(dim)]) for j in np.arange(self.n_comps_pos)]
        self.mu_neg = [np.array([2 * random() - 1 for i in np.arange(dim)]) for j in np.arange(self.n_comps_neg)]
        self.sig_pos = [spd(dim) for j in np.arange(self.n_comps_pos)]
        self.sig_neg = [spd(dim) for j in np.arange(self.n_comps_neg)]
        self.p_pos = dirichlet(np.ones(self.n_comps_pos)).rvs([])
        self.p_neg = dirichlet(np.ones(self.n_comps_neg)).rvs([])
        self.alpha = random()

    def computePNDataMetrics(self):
        epsilon = 10 ** -7
        n = 10000
        _, x, y, pos, neg, dg = self.generatePNData(n, n)
        posterior_pos = dg.pn_posterior_balanced(pos)
        irreducibility = np.mean(np.cast['int32'](posterior_pos > 1-epsilon).flatten())
        posterior_x = dg.pn_posterior_balanced(x)
        fpr, tpr, thresholds = metrics.roc_curve(y, posterior_x)
        aucpn = metrics.auc(fpr, tpr)
        #pdb.set_trace()
        return {'aucpn': aucpn, 'irreducibility': irreducibility}

    def createDataGenerator(self, n_pos, n_ul):
        dg = MVNormalMixDG(self.mu_pos, self.sig_pos, self.p_pos, self.mu_neg, self.sig_neg, self.p_neg, self.alpha,
                         n_pos, n_ul)
        return dg

    def generatePNData(self, n_pos, n_neg):
        dg = self.createDataGenerator(500, 2000)
        pos = dg.data_pos(n_pos)
        neg = dg.data_neg(n_neg)
        y = np.concatenate((np.ones([n_pos, 1]), np.zeros([n_neg, 1])), axis=0)
        x = np.concatenate((pos, neg), axis=0)
        xy = np.concatenate((x, y), axis=1)
        return xy, x, y, pos, neg, dg

    def perturb2Irreducibility(self, irr_range):
        metricsPN = self.computePNDataMetrics()
        if irr_range[0] <= metricsPN['irreducibility'] <= irr_range[1]:
            return
        sigma_flag = random( ) > 0.5
        if metricsPN['irreducibility'] < irr_range[0]:
            if sigma_flag:
                self.decrease_covar(irr_range)
            else:
                self.move_mean_out(irr_range)
        else:
            if sigma_flag:
                self.increase_covar(irr_range)
            else:
                if self.equalMeans():
                    self.alignCovar(irr_range)
                else:
                    self.move_mean_in(irr_range)
        metricsPN = self.computePNDataMetrics()
        if irr_range[0] <= metricsPN['irreducibility'] <= irr_range[1]:
            return
        else:
            self.perturb2Irreducibility(irr_range)

    def increase_covar(self, irr_range):
        i_pos = randint(0, self.n_comps_pos - 1)
        sig = self.sig_pos[i_pos]
        sig_ratio = self.sigmaRatio(sig)
        dim_float = np.cast['float32'](self.dim)
        if all(sig_ratio <= 2):
            up = 1.1
            while up > 1.001:
                self.sig_pos[i_pos] = up * sig
                metricsPN = self.computePNDataMetrics()
                if metricsPN['irreducibility'] < irr_range[0]:
                    up = 1 + (up-1)/2
                else:
                    break

    def decrease_covar(self, irr_range):
        i_pos = randint(0, self.n_comps_pos - 1)
        sig = self.sig_pos[i_pos]
        sig_ratio = self.sigmaRatio(sig)
        dim_float = np.cast['float32'](self.dim)
        if all(sig_ratio > 0.5):
            up = 0.5
            while up < .99:
                self.sig_pos[i_pos] = up * sig
                metricsPN = self.computePNDataMetrics( )
                if metricsPN['irreducibility'] > irr_range[1]:
                    up = 1 - (1 - up) / 2
                else:
                    break

    def move_mean_out(self, irr_range):
        i_pos, _ = self.componentIrreducibilitySampling()
        #pdb.set_trace()
        mu_pos = self.mu_pos[i_pos]
        i_neg, _ = self.closestNegComp(mu_pos)
        mu_neg = self.mu_neg[i_neg]
        delta = mu_pos - mu_neg
        self.mu_pos[i_pos] = self.mu_pos[i_pos] + 0.1 * delta
        metricsPN = self.computePNDataMetrics( )

        if metricsPN['irreducibility'] > irr_range[1]:
            k = 0
            lower = self.mu_pos[i_pos] - delta
            upper = self.mu_pos[i_pos]
            while ((metricsPN['irreducibility'] > irr_range[1]) or (metricsPN['irreducibility'] < irr_range[0])) and (k < 5):
                self.mu_pos[i_pos] =  (lower + upper)/2
                metricsPN = self.computePNDataMetrics( )
                if irr_range[0] <= metricsPN['irreducibility'] <= irr_range[1]:
                    break
                if metricsPN['irreducibility'] > irr_range[1]:
                    upper = self.mu_pos[i_pos]
                else:
                    lower = self.mu_pos[i_pos]
                k = k + 1

    def move_mean_in(self, irr_range):
        i_pos = randint(0, self.n_comps_pos - 1)
        upper = self.mu_pos[i_pos]
        i_neg, _ = self.closestNegComp(upper)
        lower = self.mu_neg[i_neg]
        metricsPN = self.computePNDataMetrics( )
        k = 0
        while (metricsPN['irreducibility'] > irr_range[1]) and (k < 5):
            self.mu_pos[i_pos] = (lower + upper)/2
            metricsPN = self.computePNDataMetrics( )
            if irr_range[0] <= metricsPN['irreducibility'] <= irr_range[1]:
                break
            if metricsPN['irreducibility'] > irr_range[1]:
                upper = self.mu_pos[i_pos]
            else:
                lower = self.mu_pos[i_pos]
            k = k + 1

    def align_covar(self, irr_range):
        i_pos = randint(0, self.n_comps_pos - 1)
        sig_pos = self.sig_pos[i_pos]
        i_neg, _ = self.closestNegComp(self.mu_pos[i_pos])
        sig_neg = self.sig_neg[i_neg]
        metricsPN = self.computePNDataMetrics( )
        k = 0
        up = 1.0
        low = 0.0
        while (metricsPN['irreducibility'] > irr_range[1]) and (k < 5):
            a = (up + low)/2.0
            self.sig_pos[i_pos] = a * sig_pos + (1-a) * sig_neg
            metricsPN = self.computePNDataMetrics( )
            if irr_range[0] <= metricsPN['irreducibility'] <= irr_range[1]:
                break
            if metricsPN['irreducibility'] > irr_range[1]:
                up = a
            else:
                low = a
            k = k + 1

    def equalMeans(self):
        epsilon = 10**-1
        ix_delta = [self.closestNegComp(mu) for mu in self.mu_pos]
        ix_delta = list(zip(*ix_delta))
        delta = ix_delta[1]
        sum = np.sum(delta, axis=0)
        return sum < epsilon

    def closestNegComp(self, mean):
        delta = np.array([np.sum((mean-mu)**2, axis=0) for mu in self.mu_neg])
        ix = np.argmin(delta, axis=0)
        return ix, delta[ix]

    def sigmaRatio(self, sigma):
        det = np.linalg.det(sigma)
        return np.array([det/np.linalg.det(sig_neg) for sig_neg in self.sig_neg])


    def componentIrreducibilitySampling(self):
        dg = self.createDataGenerator(500, 2000)
        comp_irr = [np.mean(dg.pn_posterior_balanced(comp.rvs(size=500)), axis=0) for comp in dg.components_pos]
        comp_irr = np.array(comp_irr)
        p = (1-comp_irr)/np.sum(1-comp_irr, axis=0)
        ix = np.random.choice(self.n_comps_pos, 1, p=p)
        ix = np.reshape(ix, newshape=())
        return ix, comp_irr

In [ ]:
# export
class NormalMixParameters:

    def __init__(self, dim, max_comps, quiet=False):
        self.dim = dim
        self.max_comps = max_comps
        #self.n_comps_pos = randint(1, max_comps)
        #self.n_comps_neg = randint(1, max_comps)
        self.n_comps_pos = max_comps
        self.n_comps_neg = max_comps
        self.mu_pos = list()
        self.mu_neg = list()
        for i in np.arange(max(self.n_comps_pos, self.n_comps_neg)):
            mu = np.array([16/np.sqrt(self.dim) * random() - 8/np.sqrt(self.dim) for i in np.arange(self.dim)])
            if i < self.n_comps_pos:
                self.mu_pos.append(mu)
            if i < self.n_comps_neg:
                self.mu_neg.append(mu)
        #self.mu_pos = [np.zeros(dim) for j in np.arange(self.n_comps_pos)]
        #self.mu_neg = [np.zeros(dim) for j in np.arange(self.n_comps_neg)]
        self.sig_pos = [np.identity(dim) for j in np.arange(self.n_comps_pos)]
        self.sig_neg = [np.identity(dim) for j in np.arange(self.n_comps_neg)]
        self.p_pos = dirichlet(np.ones(self.n_comps_pos)).rvs([])
        #self.p_neg = dirichlet(np.ones(self.n_comps_neg)).rvs([])
        self.p_neg = self.p_pos
        #self.changeInfo = {'changed': False, 'positive': True, 'mu': True, 'ix':0, 'oldvalue': self.mu_pos[0]}
        self.changeInfo = {'changed': False}
        self.alpha = random()
        self.quiet = quiet

    def computePNDataMetrics(self):
        epsilon = 0.05
        n = 10000
        _, x, y, pos, neg, dg = self.generatePNData(n, n)
        posterior_pos = dg.pn_posterior_balanced(pos)
        irreducibility = np.mean(np.cast['int32'](posterior_pos > 1-epsilon).flatten())
        posterior_x = dg.pn_posterior_balanced(x)
        fpr, tpr, thresholds = metrics.roc_curve(y, posterior_x)
        aucpn = metrics.auc(fpr, tpr)
        #pdb.set_trace()
        return {'aucpn': aucpn, 'irreducibility': irreducibility}

    def createDataGenerator(self, n_pos, n_ul):
        dg = MVNormalMixDG(self.mu_pos, self.sig_pos, self.p_pos, self.mu_neg, self.sig_neg, self.p_neg, self.alpha,
                         n_pos, n_ul)
        return dg

    def generatePNData(self, n_pos, n_neg):
        dg = self.createDataGenerator(50, 500)
        pos = dg.data_pos(n_pos)
        neg = dg.data_neg(n_neg)
        y = np.concatenate((np.ones([n_pos, 1]), np.zeros([n_neg, 1])), axis=0)
        x = np.concatenate((pos, neg), axis=0)
        xy = np.concatenate((x, y), axis=1)
        #pdb.set_trace()
        return xy, x, y, pos, neg, dg

    def perturb2SatisfyMetrics(self, irr_range, aucpn_range):
        irr_mid = np.mean(irr_range, axis=0)
        aucpn_min = min_aucpn(irr_mid)
        # if aucpn_range[0] < aucpn_min:
        #    raise ValueError('Irreducibility range and AUCPN range are not compatible:\n',
        #                      'AUCPN should be above', aucpn_min, 'for midpoint irreducibility of', irr_mid)
        while not self.isMetricSatisfied(irr_range, aucpn_range):
            self.markRandomParForChange()
            #print(self.changeInfo)
            if self.muMarked():
                self.perturbMu(irr_range, aucpn_range)
            else:
                if self.pMarked():
                    self.perturbProportion(irr_range, aucpn_range)
                else:
                    if random() <= 1:
                        self.perturbSigmaShape(irr_range, aucpn_range)
                    else:
                        self.perturbSigmaScale(irr_range, aucpn_range)
            self.commitChange()

    def perturbMu(self, irr_range, aucpn_range):
        if not self.quiet:
            print('Mu Perturb')
        c = 0.1
        delta = np.array([2 * random( ) - 1 for i in np.arange(self.dim)])
        delta = c * delta/np.linalg.norm(delta)
        mu = self.getMarkedParOldValue()
        up = 1.0
        self.proposeChange(mu + up * delta)
        while not self.isMetricUBSatisfied(irr_range, aucpn_range):
            up = up/2
            self.proposeChange(mu + up * delta)

    def perturbSigmaShape(self, irr_range, aucpn_range):
        if not self.quiet:
            print('Sigma Shape Perturb')
        newsigma = spd(self.dim)
        sigma = self.getMarkedParOldValue()
        a = 0.1
        self.proposeChange((1-a) * sigma + a * newsigma)
        while not self.isMetricUBSatisfied(irr_range, aucpn_range):
            a = a/2
            self.proposeChange((1-a) * sigma + a * newsigma)

    def perturbSigmaScale(self, irr_range, aucpn_range):
        if not self.quiet:
            print('Sigma Scale Perturb')
        sigma = self.getMarkedParOldValue()
        a = 1.5
        self.proposeChange(a * sigma)
        while not (self.isMetricUBSatisfied(irr_range, aucpn_range) and self.acceptableSigma(a * sigma)):
            a = 1 + (a - 1)/2
            #print(a)
            #print('metric:', self.isMetricUBSatisfied(irr_range, aucpn_range))
            #print('acceptable Sigma:', self.acceptableSigma(a * sigma) )
            self.proposeChange(a * sigma)

    def perturbProportion(self, irr_range, aucpn_range):
        if not self.quiet:
            print('Perturb Proportion')
        prop = self.getMarkedParOldValue( )
        a = 0.25
        if self.changeInfo['is_positive']:
            prop_1 = dirichlet(np.ones(self.n_comps_pos)).rvs([])
        else:
            prop_1 = dirichlet(np.ones(self.n_comps_neg)).rvs([])
        new_prop = (1 - a) * prop + a * prop_1
        self.proposeChange(new_prop)
        while not (self.isMetricUBSatisfied(irr_range, aucpn_range)):
            a = a/2
            new_prop = (1 - a) * prop + a * prop_1
            # print(a)
            self.proposeChange(new_prop)

    def muMarked(self):
        return self.changeInfo['is_mu']

    def pMarked(self):
        return self.changeInfo['is_proportion']

    def acceptableSigma(self, sigma):
        det = np.linalg.det(sigma)
        ratios = np.array([det/np.linalg.det(sig) for sig in self.sig_pos + self.sig_neg])
        if not self.quiet:
            print(ratios)
        ratios[:] = 1
        return all(ratios > 0.25)

    def isMetricSatisfied(self, irr_range, aucpn_range):
        metrics = self.computePNDataMetrics()
        irr_satisfied = irr_range[0] <= metrics['irreducibility'] <= irr_range[1]
        auc_satisfied = aucpn_range[0] <= metrics['aucpn'] <= aucpn_range[1]
        if not self.quiet:
            print(metrics)
        return irr_satisfied and auc_satisfied

    def isMetricUBSatisfied(self, irr_range, aucpn_range):
        metrics = self.computePNDataMetrics()
        irr_satisfied = metrics['irreducibility'] <= irr_range[1]
        auc_satisfied = metrics['aucpn'] <= aucpn_range[1]
        return irr_satisfied and auc_satisfied

    def proposeChange(self, newValue):
        self.changeInfo['changed'] = True
        V = SN(**self.changeInfo)
        self.updatePar(V.is_positive, V.is_mu, V.is_proportion, V.ix, newValue)

    def commitChange(self):
        self.changeInfo = {'changed': False}

    def updatePar(self, is_positive, is_mu, is_proportion, ix, newValue):
        if is_positive:
            if is_mu:
                self.mu_pos[ix] = newValue
            else:
                if is_proportion:
                    self.p_pos = newValue
                else:
                    self.sig_pos[ix] = newValue
        else:
            if is_mu:
                self.mu_neg[ix] = newValue
            else:
                if is_proportion:
                    self.p_neg = newValue
                else:
                    self.sig_neg[ix] = newValue

    def markRandomParForChange(self):
        if self.changeInfo['changed']:
            raise ValueError('Attempting to change a new parameter before committing the previous one')
        is_positive = random() < 0.5
        rr = random()
        is_mu = rr < 1.0/3.0
        is_proportion = 1.0/3.0 <= rr <= 2.0/3.0
        ix = np.nan
        if is_positive:
            ix = randint(0, self.n_comps_pos - 1)
            if is_mu:
                value = self.mu_pos[ix]
            else:
                if is_proportion:
                    value = self.p_pos
                else:
                    value = self.sig_pos[ix]
        else:
            ix = randint(0, self.n_comps_neg - 1)
            if is_mu:
                value = self.mu_neg[ix]
            else:
                if is_proportion:
                    value = self.p_neg
                else:
                    value = self.sig_neg[ix]

        self.changeInfo.update({'is_positive': is_positive, 'is_mu': is_mu, 'is_proportion': is_proportion, 'ix': ix, 'oldValue': value})

    def getMarkedParOldValue(self):
        return self.changeInfo['oldValue']

    def revert2OldValue(self):
        V = SN(**self.changeInfo)
        self.updatePar(V.is_positive, V.is_mu, V.ix, V.oldValue)
        self.changeInfo['changed'] = False
        return

def min_aucpn(irreducibility):
    return irreducibility + (1-irreducibility)/2

In [ ]:
# export
def GaussianMixtureDataGenerator(dim, n_comps, aucpn_range, n_p, n_u, irreducibility_range=[0.1,1]):
    NMix = NormalMixParameters(dim, n_comps,quiet=True)
    NMix.perturb2SatisfyMetrics(irreducibility_range, aucpn_range)
    dg = NMix.createDataGenerator(n_p, n_u)
    return dg

In [ ]:
dim = 10
n_comps = 5
aucpn_range = [0.5, 0.85]
n_p = 5000
n_u = 20000
dg = GaussianMixtureDataGenerator(dim, n_comps, aucpn_range, n_p, n_u)
x, y, y_p = dg.pu_data()

/home/dz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
x.shape, y.shape, y_p.shape

((25000, 10), (25000, 1), (25000, 1))